In [1]:
#Import Tensorflow Version, Google Colab Only
%tensorflow_version 2.x

In [2]:
#Mount the Notebook to Drive to Access Files
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
#Imports libraries needed
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import cv2 as cv2
import os as os
import h5py as h5py
import tensorflow as tf
from tensorflow import keras
from tqdm import tqdm #gives the status of a loop (important if we have large amounts of data and need to see the progress)

#Tensorflow Imports
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator #used for data augmentation
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, ZeroPadding2D,\
     Flatten, BatchNormalization, AveragePooling2D, Dense, Activation, Add 
from tensorflow.keras.models import Model
from tensorflow.keras import activations
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

In [4]:
#Loads the images from image preproccessing
testing_data = np.load('/content/gdrive/My Drive/Colab Notebooks/compressed_image_arrays/testing_array.npy', allow_pickle=True)
training_data = np.load('/content/gdrive/My Drive/Colab Notebooks/compressed_image_arrays/augmented_training_array.npy', allow_pickle=True)
validation_data = np.load('/content/gdrive/My Drive/Colab Notebooks/compressed_image_arrays/augmented_validation_array.npy', allow_pickle=True)

testing_labels = np.load('/content/gdrive/My Drive/Colab Notebooks/compressed_image_arrays/testing_labels.npy', allow_pickle=True)
training_labels = np.load('/content/gdrive/My Drive/Colab Notebooks/compressed_image_arrays/augmented_training_labels.npy', allow_pickle=True)
validation_labels = np.load('/content/gdrive/My Drive/Colab Notebooks/compressed_image_arrays/augmented_validation_labels.npy', allow_pickle=True)

In [5]:
#one hot encode the labels so that their dimensions fit the output
def one_hot_encode(labels):
  x = 0
  for y in labels:
    if y =='cataract':
      labels[x] = 0
    if y == 'hypertensive_retinopathy':
      labels[x] = 1
    if y == 'glaucoma':
      labels[x] = 2
    if y == 'diabetic_retinopathy_1':
      labels[x] = 3
    if y == 'diabetic_retinopathy_2':
      labels[x] = 4
    if y == 'diabetic_retinopathy_3':
      labels[x] = 5
    if y == 'diabetic_retinopathy_4':
      labels[x] = 6
    if y == 'normal':
      labels[x] = 7
    x = x+1
  new_labels = to_categorical(labels)
  return new_labels


testing_labels = one_hot_encode(testing_labels)
training_labels = one_hot_encode(training_labels)
validation_labels = one_hot_encode(validation_labels)

In [6]:
#Create ImageDataGenerator for real-time augmentation
#We only augment the training data because we train with that data
train_data_gen = ImageDataGenerator(zoom_range=0.2, 
                                   width_shift_range=0.5, 
                                   height_shift_range = 0.3, 
                                   rotation_range=30)
                                   #horizontal_flip=True,
                                   #vertical_flip=True,

valid_data_gen = ImageDataGenerator(zoom_range=0.2, 
                                   width_shift_range=0.2, 
                                   height_shift_range = 0.2, 
                                   rotation_range=30)

training_data = train_data_gen.flow(training_data, training_labels, batch_size = 32)
validation_data = valid_data_gen.flow(validation_data, validation_labels, batch_size = 32)

In [7]:
def res_identity(x, filters): 
  ''' resnet block where dimension doesnot change.
  The skip connection is just simple identity conncection
  we will have 3 blocks and then input will be added
  '''
  x_skip = x # this will be used for addition with the residual block 
  f1, f2 = filters

  #first block 
  x = Conv2D(f1, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=l2(0.001))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  #second block # bottleneck (but size kept same with padding)
  x = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=l2(0.001))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  # third block activation used after adding the input
  x = Conv2D(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=l2(0.001))(x)
  x = BatchNormalization()(x)
  # x = Activation(activations.relu)(x)

  # add the input 
  x = Add()([x, x_skip])
  x = Activation(activations.relu)(x)

  return x

def res_conv(x, s, filters):
  '''
  here the input size changes, when it goes via conv blocks
  so the skip connection uses a projection (conv layer) matrix
  ''' 
  x_skip = x
  f1, f2 = filters

  # first block
  x = Conv2D(f1, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_regularizer=l2(0.001))(x)
  # when s = 2 then it is like downsizing the feature map
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  # second block
  x = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=l2(0.001))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  #third block
  x = Conv2D(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=l2(0.001))(x)
  x = BatchNormalization()(x)

  # shortcut 
  x_skip = Conv2D(f2, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_regularizer=l2(0.001))(x_skip)
  x_skip = BatchNormalization()(x_skip)

  # add 
  x = Add()([x, x_skip])
  x = Activation(activations.relu)(x)

  return x

def resnet50():

  input_im = Input(shape=(224,224,3)) # cifar 10 images size
  x = ZeroPadding2D(padding=(3, 3))(input_im)

  # 1st stage
  # here we perform maxpooling, see the figure above

  x = Conv2D(64, kernel_size=(7, 7), strides=(2, 2))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)
  x = MaxPooling2D((3, 3), strides=(2, 2))(x)

  #2nd stage 
  # frm here on only conv block and identity block, no pooling

  x = res_conv(x, s=1, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
##152
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))

  # 3rd stage

  x = res_conv(x, s=2, filters=(128, 512))
  x = res_identity(x, filters=(128, 512))
  x = res_identity(x, filters=(128, 512))
  x = res_identity(x, filters=(128, 512))
##152
  x = res_identity(x, filters=(128, 512))
  x = res_identity(x, filters=(128, 512))
  x = res_identity(x, filters=(128, 512))
  x = res_identity(x, filters=(128, 512))

  # 4th stage

  x = res_conv(x, s=2, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))
##152
  x = res_identity(x, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))

  # 5th stage

  x = res_conv(x, s=2, filters=(512, 2048))
  x = res_identity(x, filters=(512, 2048))
  x = res_identity(x, filters=(512, 2048))
#152
  x = res_identity(x, filters=(512, 2048))
  x = res_identity(x, filters=(512, 2048))
  x = res_identity(x, filters=(512, 2048))


  # ends with average pooling and dense connection

  x = AveragePooling2D((2, 2), padding='same')(x)

  x = Flatten()(x)
  x = Dense(8, activation='softmax', kernel_initializer='he_normal')(x) #multi-class

  # define the model 

  model = Model(inputs=input_im, outputs=x, name='Resnet50')

  return model

In [8]:
#combines the whole model
resnet50_model = resnet50()

In [9]:
#prints out a whole summary of the model
#resnet50_model.summary()

In [10]:
#compiles the resnet model
resnet50_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=1e-3), 
                       metrics=['acc'])

In [11]:
#build the callbacks

#checkpoint callback
checkpoint_path = "/content/gdrive/My Drive/Colab Notebooks/checkpoints_resnet_fundus/training_batch_7_21_2020/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
print(checkpoint_dir)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

resnet50_model.save_weights(checkpoint_path.format(epoch=0))

#learning rate decay callback
def lrdecay(epoch):
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    return lr

lrdecay = tf.keras.callbacks.LearningRateScheduler(lrdecay) # learning rate decay  

def earlystop(mode):
  if mode=='acc':
    estop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=15, mode='max')
  elif mode=='loss':
    estop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, mode='min')
  return estop


/content/gdrive/My Drive/Colab Notebooks/checkpoints_resnet_fundus/training_batch_7_21_2020


In [12]:
batch_size = 128

In [13]:
resnet_train = resnet50_model.fit_generator(training_data, 
                                            epochs=250, 
                                            steps_per_epoch= 2485 / batch_size, 
                                            validation_steps= 3332 / batch_size, 
                                            validation_data = validation_data, 
                                            callbacks=[lrdecay],)
                                            #cp_callback, 

#okay, so the error on the bottom has something to do with this line:  x = Dense(8, activation='softmax', kernel_initializer='he_normal')(x) #multi-class
#the "8" parameter is correct because there are (none, 8) outputs, so the input must be (none, 1)
#how should i fix this?
#should I one hot encode the training, validation and testing labels to so that it will be size 8?
#YUP one hot encoding fix it

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/250
20/19 [==============================] - 20s 1s/step - loss: 53.8792 - acc: 0.3578 - val_loss: 38.1822 - val_acc: 0.5324 - lr: 0.0010
Epoch 2/250
20/19 [==============================] - 17s 828ms/step - loss: 41.2403 - acc: 0.4797 - val_loss: 37.6479 - val_acc: 0.5220 - lr: 0.0010
Epoch 3/250
20/19 [==============================] - 17s 838ms/step - loss: 40.3198 - acc: 0.3922 - val_loss: 36.9133 - val_acc: 0.5000 - lr: 0.0010
Epoch 4/250
20/19 [==============================] - 17s 830ms/step - loss: 39.4957 - acc: 0.4234 - val_loss: 36.2046 - val_acc: 0.0880 - lr: 0.0010
Epoch 5/250
20/19 [==============================] - 17s 830ms/step - loss: 37.2713 - acc: 0.4344 - val_loss: 35.6229 - val_acc: 0.0741 - lr: 0.0010
Epoch 6/250
20/19 [==============================] - 17s 841ms/step - loss: 35.9096 - acc: 0.4375 - val_loss: 34.6268 - val_acc: 0.0775 - lr: 0.0010
Epoch 7/250
20/19 [==============